# Forest Fire Detection with transfer learning using InceptionV3

First we import all the necessary libraries.

In [1]:
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import optimizers
from keras.models import load_model
from keras.applications import InceptionV3
from keras import layers
from keras import Model
import matplotlib.pyplot as plt
%matplotlib inline

## Getting the Data

Now we declare the path for our training and testing data. In this case, we are loading from our current directory, assuming the data is in a folder in this same level called data/

In [2]:
train_dir = "data/Training"
test_dir = "data/Test"

SIZE = 299

Now we create the train and test data generators. For the train data generator we apply data augmentation by providing a width and height shift range, as well as a zoom range and horizontal split. We also specify that we want 20% of the training data as validation set.

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                zoom_range=0.1,
                                horizontal_flip=True,
                                validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

We now create the train, validation and test generators by specifying the directories.

In [4]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                target_size=(SIZE, SIZE), 
                                                color_mode='rgb',
                                                batch_size=20,
                                                subset='training',
                                                class_mode='binary')

val_generator = train_datagen.flow_from_directory(train_dir,
                                                target_size=(SIZE, SIZE), 
                                                color_mode='rgb',
                                                batch_size=20,
                                                subset='validation',
                                                class_mode='binary')

test_generator = test_datagen.flow_from_directory(test_dir,
                                                target_size=(SIZE, SIZE),
                                                color_mode='rgb',
                                                batch_size=20,
                                                class_mode='binary')

Found 31517 images belonging to 2 classes.
Found 7878 images belonging to 2 classes.
Found 8617 images belonging to 2 classes.


## Creating the Model

Now we first load the pre-trained ResNet50 model from the keras library with the imagenet weights

In [5]:
transfer_model = InceptionV3(include_top=True,
                        weights="imagenet",
                        classes=1000)
input = transfer_model.inputs

We now add our own top layers, and specify that all, except our top layers are not trainable, basically freezing the weights

In [11]:
x = transfer_model.get_layer("avg_pool").output
x = layers.Flatten(name='flatten')(x)
x = layers.Dense(4096, activation='relu')(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(4096, activation='relu')(x)
output_layer = layers.Dense(1, activation='sigmoid')(x)

classifier = Model(input, output_layer)

for layer in classifier.layers[:-5]:
    layer.trainable = False

classifier.summary()

60[0][0]     
__________________________________________________________________________________________________
activation_63 (Activation)      (None, 17, 17, 192)  0           batch_normalization_63[0][0]     
__________________________________________________________________________________________________
activation_68 (Activation)      (None, 17, 17, 192)  0           batch_normalization_68[0][0]     
__________________________________________________________________________________________________
activation_69 (Activation)      (None, 17, 17, 192)  0           batch_normalization_69[0][0]     
__________________________________________________________________________________________________
mixed7 (Concatenate)            (None, 17, 17, 768)  0           activation_60[0][0]              
                                                                 activation_63[0][0]              
                                                                 activation_68[0][0]           

We now compile our classifier with binary crossentropy as loss function and Adam optimizer with 0.001 learning rate.

In [12]:
classifier.compile(loss="binary_crossentropy",
                optimizer=optimizers.Adam(lr=0.001),
                metrics=["accuracy"])

## Training the Model

Now we train the model adding some callbacks for early stopping and to save the model every epoch.

In [13]:
model_path = 'models/inceptionv3_model.h5'

callbacks = [
    EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True),
    ModelCheckpoint(filepath=model_path, save_best_only=True, monitor='val_accuracy'),
]

history = classifier.fit(train_generator, 
                        epochs=5, 
                        batch_size=40,
                        validation_data=val_generator,
                        callbacks=callbacks)

Epoch 1/5
1576/1576 [==============================] - 909s 574ms/step - loss: 0.2323 - accuracy: 0.9525 - val_loss: 0.1034 - val_accuracy: 0.9667
Epoch 2/5
1576/1576 [==============================] - 853s 541ms/step - loss: 0.0434 - accuracy: 0.9850 - val_loss: 0.2266 - val_accuracy: 0.9412
Epoch 3/5
1067/1576 [===================>..........] - ETA: 3:45 - loss: 0.0342 - accuracy: 0.9875

KeyboardInterrupt: 

We now graph the results of the training.

In [1]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

NameError: ignored

## Evaluating the Model

Now we run the evaluation on the test set and see the result.

In [ ]:
result = classifier.evaluate(test_generator)
print(f"Test loss: {result[0]}, Test accuracy: {result[1]}")

431/431 [==============================] - 407s 945ms/step - loss: 0.8416 - accuracy: 0.6393
Test loss: 0.8416192531585693, Test accuracy: 0.6393176317214966
